In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn import metrics
import numpy as npw
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.preprocessing import MinMaxScaler





# Read the CSV file
data = pd.read_csv("../data/data/aapl_raw_data.csv")
data.shape
data.tail(1)

data = data.iloc[:10747]

data.tail(1)


data.isnull().sum()
data=data.fillna(0)  # Filling null values with zero
data.isnull().sum()



#data["date"] = data["date"].astype(float)
data["open"] = data["open"].astype(float)
data["high"] = data["high"].astype(float)
data["low"] = data["low"].astype(float)
data["volume"] = data["volume"].astype(float)
data["adjusted_close"] = data["adjusted_close"].astype(float)
data["change_percent"] = data["change_percent"].astype(float)
data["avg_vol_20d"] = data["avg_vol_20d"].astype(float)

data["close"] = data["close"].astype(float)

#print(data.tail(1))

In [3]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Convert data to numpy arrays
x_data = data[['open', 'high', 'low', 'volume', 'adjusted_close', 'change_percent', 'avg_vol_20d']].values
y_data = data["close"].values.reshape(-1, 1)  # Reshape y_data if it's 1D

# Initialize the scalers
scaler_x = StandardScaler()
scaler_y = StandardScaler()

# Fit the scaler to the training data and transform both training and test data
x_data_scaled = scaler_x.fit_transform(x_data)
y_data_scaled = scaler_y.fit_transform(y_data)

# Convert scaled data to Tensors
x_feature_tensors = torch.tensor(x_data_scaled, dtype=torch.float32)
y_feature_tensors = torch.tensor(y_data_scaled, dtype=torch.float32)

# Split the training data into training and temporary sets
x_train, x_temp, y_train, y_temp = train_test_split(x_feature_tensors, y_feature_tensors, test_size=0.2, random_state=42)

# Split the temporary set into validation and test sets
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

# Convert scaled labels back to numpy arrays
y_train = y_train.numpy()
y_val = y_val.numpy()
y_test = y_test.numpy()



# Now you can use x_train_scaled, x_val, x_test, y_train_scaled, y_val_scaled, y_test_scaled for your model training


In [4]:

learning_rate = 0.001
window_size = 50
test_window_size = 50
hidden_dim = 64
n_layers = 4
batch_evaluation_frequency = 10
epochs = len(x_train) - window_size
batch_size = 1
input_size = x_train.shape[1]  # Input size based on your dataset
output_size = 1  # Output size (for regression task)

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_dim, n_layers, output_size):
        super(LSTMModel, self).__init__()

        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.lstm = nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x, hidden):
        batch_size = x.size(0)
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out[:, -1, :])
        return out, hidden

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        return hidden


In [5]:
import torch
import torch.optim as optim
import torch.nn as nn
from sklearn.model_selection import train_test_split
import random
import pandas as pd

# Assuming x_train_scaled, y_train_scaled, x_val_scaled, y_val_scaled, x_test_scaled, y_test_scaled,
# input_size, output_size, test_window_size, scaler are available


# Hyperparameter search space
search_space = {
    'learning_rate': [0.00005],
    'window_size': [ 5],
    'hidden_dim': [256],
    'n_layers': [11],
    'batch_evaluation_frequency': [4]
}

#Best Parameters: {'learning_rate': 5e-05, 'window_size': 5, 'hidden_dim': 256, 'n_layers': 11, 'batch_evaluation_frequency': 4, 'test_loss': 0.41702309250831604}
#              'n_layers': 11, 'batch_evaluation_frequency': 4, 'test_loss': 6463.20556640625}


# Number of random search iterations
num_iterations = 10

best_params = None

for _ in range(num_iterations):
    params = {
        'learning_rate': random.choice(search_space['learning_rate']),
        'window_size': random.choice(search_space['window_size']),
        'hidden_dim': random.choice(search_space['hidden_dim']),
        'n_layers': random.choice(search_space['n_layers']),
        'batch_evaluation_frequency': random.choice(search_space['batch_evaluation_frequency'])
    }

    model = LSTMModel(input_size, params['hidden_dim'], params['n_layers'], output_size)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])

    # Training using Walk-Forward Validation
    for i in range(params['window_size'], len(x_train)):
        optimizer.zero_grad()

        start_idx = i - params['window_size']
        end_idx = i
        x_window = torch.tensor(x_train[start_idx:end_idx], dtype=torch.float32)
        y_window = torch.tensor(y_train[start_idx:end_idx], dtype=torch.float32)
        x_window = x_window.view(1, params['window_size'], input_size)

        hidden = model.init_hidden(1)
        outputs, hidden = model(x_window, hidden)
        loss = criterion(outputs, y_window)

        loss.backward()
        optimizer.step()

        if i % params['batch_evaluation_frequency'] == 0:
            with torch.no_grad():
                x_val_window = torch.tensor(x_val[:params['window_size']], dtype=torch.float32)
                y_val_window = torch.tensor(y_val[:params['window_size']], dtype=torch.float32)
                x_val_window = x_val_window.view(1, params['window_size'], input_size)

                hidden = model.init_hidden(1)
                val_outputs, _ = model(x_val_window, hidden)
                val_loss = criterion(val_outputs, y_val_window)

                if i % 1000 == 0:  # Print every 1000 iterations
                    print(f"Iteration {i}, Train Loss: {loss.item()}, Validation Loss: {val_loss.item()}")



    # Test set evaluation
    with torch.no_grad():
    # Use the last 'window_size' entries from the original data as the input for the model
        x_test_window = torch.tensor(x_data_scaled[-params['window_size']:], dtype=torch.float32)
        y_test_window = torch.tensor(y_data_scaled[-params['window_size']:], dtype=torch.float32)
        x_test_window = x_test_window.view(1, params['window_size'], input_size)

        hidden = model.init_hidden(1)
        test_outputs, _ = model(x_test_window, hidden)
        test_loss = criterion(test_outputs, y_test_window)




    print(f"Test Loss: {test_loss.item()}")

    # Update the best_params if the current model performs better on the test set
    if best_params is None:
        best_params = params
        best_params['test_loss'] = test_loss.item()
    elif test_loss < best_params['test_loss']:
        best_params = params
        best_params['test_loss'] = test_loss.item()

print(f"Best Parameters: {best_params}")

C:\Users\pitr7\AppData\Local\Temp\ipykernel_6296\267311635.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_window = torch.tensor(x_train[start_idx:end_idx], dtype=torch.float32)
c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([5, 1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\pitr7\AppData\Local\Temp\ipykernel_6296\267311635.py:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_val_window = torch.tensor(x_val[:p

Iteration 1000, Train Loss: 1.0491517782211304, Validation Loss: 2.2497355937957764
Iteration 2000, Train Loss: 2.0725128650665283, Validation Loss: 2.234780788421631
Iteration 3000, Train Loss: 0.23910768330097198, Validation Loss: 2.2374267578125
Iteration 4000, Train Loss: 0.3821171820163727, Validation Loss: 2.2168617248535156
Iteration 5000, Train Loss: 0.30708590149879456, Validation Loss: 2.219841957092285
Iteration 6000, Train Loss: 0.17089402675628662, Validation Loss: 2.223947763442993
Iteration 7000, Train Loss: 0.23000116646289825, Validation Loss: 2.229879856109619
Iteration 8000, Train Loss: 0.33763909339904785, Validation Loss: 2.2214770317077637
Test Loss: 0.3516319990158081
Iteration 1000, Train Loss: 1.0487529039382935, Validation Loss: 2.2495007514953613
Iteration 2000, Train Loss: 2.072859525680542, Validation Loss: 2.234898090362549
Iteration 3000, Train Loss: 0.23985806107521057, Validation Loss: 2.2369916439056396
Iteration 4000, Train Loss: 0.380383163690567, Va

In [22]:
# Print the last 5 rows of the original data
print("Last 5 Rows of Original Data:")
print(data.tail(5))

Last 5 Rows of Original Data:
             date    open    high     low   close      volume  adjusted_close  \
10742  2023-07-25  193.33  194.44  192.92  193.62  37283200.0        193.3589   
10743  2023-07-26  193.67  195.64  193.32  194.50  47471900.0        194.2377   
10744  2023-07-27  196.02  197.20  192.55  193.22  47460200.0        192.9594   
10745  2023-07-28  194.67  196.63  194.14  195.83  48291400.0        195.5659   
10746  2023-07-31  196.06  196.49  195.26  196.45  38824100.0        196.1851   

       change_percent  avg_vol_20d  
10742            0.45   52369165.0  
10743            0.45   52206220.0  
10744           -0.66   52018390.0  
10745            1.35   52115595.0  
10746            0.32   49803320.0  


In [16]:
# Apply the same scaling operation to the last 5 rows of the original data
last_5_rows_scaled = scaler_x.transform(x_data[-5:])
print("Last 5 Rows of Original Data (Scaled):")
print(last_5_rows_scaled)





Last 5 Rows of Original Data (Scaled):
[[ 0.61575973  0.60863248  0.62957796  0.69611326  4.55759397  0.1205331
  -0.99750238]
 [ 0.61833886  0.61765745  0.63264222  1.07323779  4.5803999   0.1205331
  -0.99810086]
 [ 0.6361652   0.62938991  0.62674351  1.07280473  4.54722646 -0.2740075
  -0.99879075]
 [ 0.62592454  0.62510305  0.63892397  1.10357077  4.6148683   0.44043087
  -0.99843372]
 [ 0.63646862  0.62405014  0.64750392  0.75314813  4.63093729  0.07432564
  -1.00692654]]


In [17]:
# Print the last training window in original scale
print("Last Training Window in Original Scale:")
print(last_training_window_original_scale)

# Print the last 5 rows of original data in the same scale as the last training window
print("Last 5 Rows of Original Data in Same Scale:")
print(last_5_rows_scaled)

Last Training Window in Original Scale:
[[ 2.21983979e+01  2.28479971e+01  2.20528002e+01  4.69919920e+06
   3.42700308e-01  1.87000004e+00  2.07898037e+08]
 [ 2.30496039e+01  2.39120000e+01  2.29376033e+01  6.22159998e+06
   3.56800340e-01  8.50000004e-01  3.56143759e+08]
 [ 2.54911971e+01  2.54912027e+01  2.49311975e+01  6.70369970e+06
   3.78700292e-01 -1.69000008e+00  2.84303878e+08]
 [ 3.12479993e+01  3.31295967e+01  3.12480012e+01  1.13840055e+06
   1.13800448e-01  5.65999996e+00  2.25719198e+08]
 [ 1.87739997e+02  1.90010800e+02  1.87300398e+02  2.16101999e+07
   5.76009959e+00  2.14999995e+00  5.13932433e+08]]
Last 5 Rows of Original Data in Same Scale:
[[ 0.61575973  0.60863248  0.62957796  0.69611326  4.55759397  0.1205331
  -0.99750238]
 [ 0.61833886  0.61765745  0.63264222  1.07323779  4.5803999   0.1205331
  -0.99810086]
 [ 0.6361652   0.62938991  0.62674351  1.07280473  4.54722646 -0.2740075
  -0.99879075]
 [ 0.62592454  0.62510305  0.63892397  1.10357077  4.6148683   0.4